In [2]:
import pandas as pd
import os
import pyarrow
import openpyxl

import os
import pyarrow
import openpyxl
import pickle
from typing import Dict

from exgauster.utils import DataOfExgauster

with open('prepared_data.pkl', "rb") as f:
    exgausters_data: Dict[int, DataOfExgauster] = pickle.load(f)



C:\tools\miniconda3\envs\stal\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
x_train = pd.concat([exgausters_data[4].train, exgausters_data[4].test])

In [4]:
from merlion.utils.time_series import TimeSeries


target_seq_index = 0

max_forecast_steps = len(pd.date_range(start='2023-04-10 00:00:00', end='2023-05-01 00:00:00', freq="5Min"))
n_past = 3 * max_forecast_steps

print(max_forecast_steps)

kwargs = dict(target_seq_index=target_seq_index, max_forecast_steps=max_forecast_steps, n_past=n_past)

6049


In [5]:
from merlion.models.factory import ModelFactory


model2 = ModelFactory.create("TransformerForecaster", **kwargs)

In [6]:
# from merlion.models.ensemble.combine import Mean
#
# model3 = ModelFactory.create("ForecasterEnsemble", models=[model1, model2],
#                                  combiner=Mean())

NameError: name 'model1' is not defined

In [7]:
import torch
torch.cuda.empty_cache()
import gc
gc.collect()

model1 = ModelFactory.create(
    "ETSformerForecaster",
    batch_size=100,
    **kwargs
)


x_train_ts = TimeSeries.from_pd(x_train).align()
train_pred, train_stderr = model1.train(x_train_ts)

OutOfMemoryError: CUDA out of memory. Tried to allocate 3.46 GiB (GPU 0; 8.00 GiB total capacity; 4.64 GiB already allocated; 1.23 GiB free; 4.67 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF